In [4]:
##%overwritefile
##%file:src/compile_with_kotlinc.py
##%noruncode
    def compile_with_kotlinc(self, source_filename, 
        binary_filepath=None, 
        cflags=None, ldflags=None,
        env=None,coptions=None,magics=None):
        # coptions = ['-b', '', '-cp', '', '-j'] + cflags
        outpath=os.path.dirname(source_filename)
        sf = os.path.basename(source_filename)
        binary_filename = sf.split(".")[0]
        # if self.linkMaths:
        #     cflags = cflags + ['-lm']
        # if self.wError:
        #     cflags = cflags + ['-Werror']
        # if self.wAll:
        #     cflags = cflags + ['-Wall']
        # if self.readOnlyFileSystem:
        #     cflags = ['-DREAD_ONLY_FILE_SYSTEM'] + cflags
        # if self.bufferedOutput:
        #     cflags = ['-DBUFFERED_OUTPUT'] + cflags
        index=-1
        if coptions==None:
            coptions=[]
        for s in coptions:
            index=index+1
            if s=='-d':
                outpath=coptions[index+1]
                if not outpath.startswith('-'):
                    #剔除 -d参数和值
                    outpath=coptions.pop(index+1)
                    coptions.pop(index)
            else:
                if binary_filepath!=None:
                    outpath=binary_filepath
        args = ['kotlinc']+coptions+ ['-d', outpath]+[ source_filename]
        self.mymagics._log(' '.join((' '+ str(s) for s in args))+"\n")
        binary_filename=binary_filename.capitalize()
        binary_filename=os.path.join(outpath,binary_filename)
        binary_filename=binary_filename+"Kt" 
        # self.mymagics._log("binary_filename:"+binary_filename+"\n")
        return self.mymagics.create_jupyter_subprocess(args,env=env,magics=magics),binary_filename+".class",args


[MyPythonKernel125652] Info:src/compile_with_kotlinc.py
[MyPythonKernel125652] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/Kotlin/src/compile_with_kotlinc.py created successfully


In [5]:
##%overwritefile
##%file:src/k_exec_kotlinc_.py
##%noruncode
    def _exec_kotlinc_(self,source_filename,magics):
        self.mymagics._logln('Generating binary file')
        magics['status']='compiling'
        p,outfile,gcccmd = self.compile_with_kotlinc(
            source_filename, 
            None,
            self.mymagics.get_magicsSvalue(magics,'cflags'),
            self.mymagics.get_magicsSvalue(magics,'ldflags'),
            self.mymagics.get_magicsbykey(magics,'env'),
            self.mymagics.get_magicsSvalue(magics,'coptions'),
            magics
            )
        returncode=p.wait_end(magics)
        p.write_contents()
        magics['status']=''
        if returncode != 0:  # Compilation failed
            self.mymagics._logln(''.join((str(s) for s in ccmd)),3)
            self.mymagics._logln("Kotlin exited with code {}, the executable will not be executed".format(returncode),3)

            # delete source files before exit
            os.remove(source_filename)
            # os.remove(binary_file.name)
        return p.returncode,outfile


[MyPythonKernel125652] Info:src/k_exec_kotlinc_.py
[MyPythonKernel125652] Info:file /mnt/h/Jupyter/Myjupyter-kernel/kernel/Kotlin/src/k_exec_kotlinc_.py created successfully
